# Celestial mechanics with the `celmech` code

**Sam Hadden** (CITA)

Co-author: **Dan Tamayo** (Harvey-Mudd)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sympy import init_printing
init_printing()

# What is `celmech`?

- Code for celestial mechanics calculations
    - Calculate disturbing function expansion
    - Construct, manipulate, and integrate Hamiltonian equations of motion
    - Much more...
- Developed mainly in `python`
- Designed to work with the [`rebound`](https://github.com/hannorein/rebound) N-body code (Rein & Liu 2012)
- Symbolic mathematics with [`sympy`](https://www.sympy.org/en/index.html) (Meurer et. al. 2017)

# Where is `celmech`?

- PyPI: ``pip install celmech``
- GitHub reposity at [github.com/shadden/celmech](https://github.com/shadden/celmech) 
    - Includes lots of Jupyter notebook examples
- Documentation at [celmech.readthedocs.io](https://celmech.readthedocs.io)
- [Paper](https://ui.adsabs.harvard.edu/abs/2022AJ....164..179H/abstract) in AJ

# Basic principles

- $N$-body codes like `rebound` integrate exact equations of motion directly

- `celmech` works with approximate equations of motion derived from disturbing function expansion:
 $$
 \begin{multline}
 -\frac{Gm_im_j}{|\mathbf{r}_i - \mathbf{r}_j |} +\mathrm{indirect~terms^*}
 =\\ 
 -\frac{Gm_im_j}{a_j}
      \sum_{\bf k}     
     \sum_{\nu_1,\nu_2,\nu_3,\nu_4=0}^\infty                       
     \tilde{C}_{\bf k}^{{\nu}}(\alpha)
     s_i^{|k_5|+2\nu_1}
     s_j^{|k_6|+2\nu_2}
     e_i^{|k_3|+2\nu_3}
     e_i^{|k_4|+2\nu_4}
     \\
     \times \cos(k_1\lambda_j+k_2\lambda_i+k_3\varpi_i+k_4\varpi_j+k_5\Omega_i+k_6\Omega_j)
 \end{multline}
$$
where $s_i=\sin(I_i/2)$.

'*': coodinate-system dependent

# An example
A system of two Earth-mass planets near a 3:2 mean motion resonance

In [ ]:
import rebound
rebound_sim = rebound.Simulation()
rebound_sim.add(m=1)
rebound_sim.add(m=3e-6,P = 1, e = 0.04)
rebound_sim.add(m=3e-6,P = 3 / 2, e = 0.02,l=0.5,pomega = np.pi+1)
rebound.OrbitPlot(rebound_sim,color=True,periastron=True);
plt.show()

# Initializing a ``celmech`` model 
- Create `Poincare` and `PoincareHamiltonian` instances
- Initialize directly from a ``rebound`` simulation

In [ ]:
from celmech import Poincare, PoincareHamiltonian
poincare_particles = Poincare.from_Simulation(rebound_sim)
Hp = PoincareHamiltonian(poincare_particles)

# The ``Poincare`` class 
- Represents system in terms of canonical variable pairs:
 $$\begin{align}
 \Lambda_i&= \mu_i\sqrt{GM_ia_i} ~;~ \lambda_i \\
 (\eta_i,\kappa_i)&\approx \sqrt{\Lambda_i}e_i \times(-\sin\varpi_i,\cos\varpi_i)\\
  (\rho_i,\sigma_i)&\approx \sqrt{\Lambda_i}\sin(I_i/2) \times(-\sin\Omega_i,\cos\Omega_i)\\
 \end{align}$$

In [ ]:
type(poincare_particles)

In [ ]:
qp_pairs = poincare_particles.qp_pairs
qp_pairs

# The ``Poincare`` class 
- Stores numerical values of canonical variables

In [ ]:
Lambda1 = qp_pairs[0][1]
Lambda1, poincare_particles.qp[Lambda1]

- Also provides orbital mass, orbital elements, etc.

In [ ]:
particle = poincare_particles.particles[1]
print(particle.m,particle.a,particle.e,particle.inc)

# The ``PoincareHamiltonian`` class 
- Represents a planetary system's Hamiltonian


In [ ]:
type(Hp)

- The ``H`` attribute stores the symbolic Hamiltonian
    - Keplerian terms upon intialization 

In [ ]:
Hp.H

 - Or, in orbital elements: 
 $$
 -\frac{Gm_*m_1}{2a_1}-\frac{Gm_*m_2}{2a_2}
 $$

# The ``PoincareHamiltonian`` class 
- Stores numerical values of symbolic parameters as well

In [ ]:
Hp.N_H

# Building a Hamiltonian
- Users build up a Hamiltonian by selecting and adding disturbing function terms
- ``PoincareHamiltonian`` includes an extensive interface for specifying and adding terms
- We'll add terms for the 3:2 MMR

In [ ]:
Hp.add_MMR_terms(p=3,q=1,max_order=1,indexIn=1,indexOut=2)

## The symbolic Hamiltonian is updated with the newly-added terms

In [ ]:
Hp.H

# Building a Hamiltonian
- Equivalent in terms of orbital elements:

In [ ]:
Hp.df

# Integrating Hamilton's equations
 - Equations of motion automatically generated from the Hamiltonian
 - Integration and particle interface designed to mirror ``rebound``

In [ ]:
rebound_particles = rebound_sim.particles
celmech_particles = Hp.particles

# times to save output
times = np.linspace(0,2500 * rebound_particles[1].P,100)

# Arrays to store results
theta_rebound = np.zeros(100)
theta_celmech = np.zeros(100)

# Main integration loop
for i,t in enumerate(times):
    rebound_sim.integrate(t)
    Hp.integrate(t)
    # save resonant angle
    theta_celmech[i] = 3*celmech_particles[2].l - 2 * celmech_particles[1].l - celmech_particles[1].pomega
    theta_rebound[i] = 3*rebound_particles[2].l - 2 * rebound_particles[1].l - rebound_particles[1].pomega
    

wrap2pi = lambda x: np.mod(x+np.pi,2*np.pi)-np.pi
theta_celmech=wrap2pi(theta_celmech)
theta_rebound=wrap2pi(theta_rebound)

# Comparing $N$-body and ``celmech``
- The simple ``celmech`` model shows fair agreement with direct $N$-body integration

In [ ]:
fig,ax = plt.subplots(1,sharex=True,figsize=(8,5))
ax.plot(times,theta_rebound,color='k',lw=2,label='rebound')
ax.plot(times,theta_celmech,color='r',lw=2,label='celmech')
plt.tick_params(labelsize=15,direction='in',size=8)
ax.set_ylabel(r"$3\lambda_2-2\lambda_1-\varpi_1$",fontsize=20)
ax.yaxis.set_ticks([-np.pi,-0.5*np.pi,0,0.5*np.pi,np.pi],
                   labels=[r"$-\pi$",r"$-\pi/2$",r"$0$",r"$\pi/2$",r"$\pi$"]
                  )
ax.legend(fontsize=12)
ax.set_xlabel(r"Time [$P_\mathrm{in}$]",fontsize=20)

# Laplace-Lagrange secular theory
- Classical Laplace-Lagrange secular theory provides an approximation for the evolution of planets' eccentricities and inclinations

- Individual planets' elements $(h_i,k_i)=e_i\times (\sin\varpi_i,\cos\varpi_i)$ and $(p_i,q_i)=\sin(I_i/2)\times (\sin\varpi_i,\cos\varpi_i)$ are described as a sum of modes. I.e.,

$$ 
\begin{align}
h_i(t) &= \sum_{j=1}^N a_{ij}\sin(g_j t + \phi_j) &;~ k_i(t) = \sum_{j=1}^N a_{ij}\cos(g_j t + \phi_j)
\\
p_i(t) &=\sum_{j=1}^{N} b_{ij}\sin(s_j t + \psi_j)& ;~q_i(t)=\sum_{j=1}^{N} b_{ij}\cos(s_j t + \psi_j)
\end{align}
$$

# Laplace-Lagrange secular theory
We'll use `celmech` to construct the predictions of Laplace-Lagrange secular theory for the outer solar system.

In [ ]:
# Set up the outer solar system
sim = rebound.Simulation()
sim.units=('Msun','yr','AU')
for body in ("Sun","Jupiter","Saturn","Uranus","Neptune"):
    sim.add(body)
sim.move_to_com()
sim.integrator = 'whfast'
sim.dt = sim.particles[1].P / 20.

# Laplace-Lagrange secular theory
We could go ahead and use `celmech` to construct and integrate the secular Hamiltonian to leading order in eccentricities and inclinations:

In [ ]:
# Initialize Poincare Hamiltonian
pvars = Poincare.from_Simulation(sim)
pham = PoincareHamiltonian(pvars)

In [ ]:
# add secular all secular terms for each planet pair up to second order in e and I
for i in range(1,sim.N):
    for j in range(i+1,sim.N):
        pham.add_secular_terms(indexIn=i,indexOut=j,max_order=2)

In [ ]:
pham.df

# Laplace-Lagrange secular theory
Instead, we'll use the class `celmech.secular.LaplaceLagrangeSystem` to construct an analytic solution for a system's secular evolution

In [ ]:
from celmech.secular import LaplaceLagrangeSystem
ll_sys = LaplaceLagrangeSystem.from_Simulation(sim)

# Laplace-Lagrange secular theory
We'll use a `celmech` convenience function, `get_simarchive_integration_results` to load the results of a `rebound.Simulationarchive` file

In [ ]:
# Do integration if saved file doesn't exist
save_file = 'outer_ss.sa'
from os.path import exists
if not exists(save_file):
    Tfin = 2e6
    sim.save_to_file('outer_ss.sa',delete_file=True,interval = Tfin / 256)
    sim.integrate(Tfin)

In [ ]:
# Load a dictionary of results
from celmech.nbody_simulation_utilities import get_simarchive_integration_results as get_results
nbody_results = get_results('outer_ss.sa')

# Laplace-Lagrange secular theory
The `LaplaceLagrangeSystem.secular_solution` method computes an analytic solution for planets' secular orbital evolution at user-specified times.

In [ ]:
times = nbody_results['time']
ll_results = ll_sys.secular_solution(times)

In [ ]:
planets=("Jupiter","Saturn","Uranus","Neptune")
fig,ax= plt.subplots(4,4,figsize=(16,8),sharex=True,sharey='row')
for i in range(4):
    # eccentricity variables
    h = nbody_results['e'][i] * np.sin(nbody_results['pomega'][i])
    k = nbody_results['e'][i] * np.cos(nbody_results['pomega'][i])
    ax[i,0].plot(times/1e3,h,'k')
    ax[i,1].plot(times/1e3,k,'k')
    # inclination variables
    s = np.sin(0.5*nbody_results['inc'][i]) 
    p = s * np.sin(nbody_results['Omega'][i])
    q = s * np.cos(nbody_results['Omega'][i])
    ax[i,2].plot(times/1e3,p,'k')
    ax[i,3].plot(times/1e3,q,'k')
    
    # celmech solution
    ax[i,0].plot(times/1e3,ll_results['h'][i],'r')    
    ax[i,1].plot(times/1e3,ll_results['k'][i],'r')
    ax[i,2].plot(times/1e3,ll_results['p'][i],'r')
    ax[i,3].plot(times/1e3,ll_results['q'][i],'r')
    
    ax[i,0].set_ylabel(planets[i])
for i in range(4):
    ax[3,i].set_xlabel("Time [kyr]",fontsize=14)
    title = (r'$e_i\sin\varpi_i$',r'$e_i\cos\varpi_i$',r'$\sin(I_i/2)\sin\Omega_i$',r'$\sin(I_i/2)\cos\Omega_i$')
    ax[0,i].set_title(title[i],fontsize=14)

## But wait, there's much more!
 - **Hamiltonian mechanics**
     - Construct arbitrary Hamiltonians & integrate equations of motion
     - Canonical transformations
     - Lie series perturbation theory
 - **Secular theory**
     - Laplace-Lagrange theory (including 2nd order in mass)
     - Non-linear secular equations
         - Symplectic integration
         - Second order in mass
 - **Mean motion resonance**
     - Numerically-averaged resonance equations
         - No expansion in $e$ and $I$
         - Supports dissipative forces
 - And **more**! (resonant chains, TTVs, frequency analysis, AMD,...)